# Summary File Generation

This notebook generates the summary file required for the first page of the feedback-lens app.
It assume the `csv` exist in the same folder it would create the output file.

With the scope of the Hackathon we used 3 services to generate the summary file:

- cosmosdb.csv
- adf.csv
- aks.csv

In [ ]:
#r "nuget: CsvHelper"
#r "nuget: DotNetEnv, 2.5.0"

In [2]:
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;
using CsvHelper;
using CsvHelper.Configuration;
// using Newtonsoft.Json;
using System.Text.Json;  // For JSON serialization/deserialization
using System.Text.Json.Serialization;
using DotNetEnv;

In [ ]:
static string _configurationFile = @"../../configuration/.env";
Env.Load(_configurationFile);
string dataRoot = Environment.GetEnvironmentVariable("DB_ROOT_FOLDER") ?? "DB_ROOT_FOLDER not found";

// set the service name: aks, adf or cosmos
string serviceName = "adf";

Console.WriteLine($"Working directory: {dataRoot} using service file: {serviceName}.csv");

In [11]:
 # load "../FeedbackApi/Models/CSVFeedbackRecord.cs"
 # load "../FeedbackApi/Models/ServiceHighlight.cs"

## Creating the summary

- Per csv file, we run this short summarization based on meta data

In [12]:

var config = new CsvConfiguration(System.Globalization.CultureInfo.InvariantCulture)
{
    HasHeaderRecord = true,
};

In [14]:
public string GenerateSummary(string service){
    var filePath = $"{dataRoot}/{service}.csv";
    using (var reader = new StreamReader(filePath))
    using (var csv = new CsvReader(reader, config))
    {
        var records = csv.GetRecords<CSVFeedbackRecord>().ToList();

        // Group by ServiceTree_Name and calculate metrics
        var serviceHighlights = records
            .GroupBy(r => r.ServiceName)
            .Select(g =>
            {
                var totalFeedback = g.Count();
                var feedbackByType = g.GroupBy(r => r.Type)
                                    .Select(t => new FeedbackTypeSummary
                                    {
                                        Type = t.Key,
                                        Count = t.Count()
                                    })
                                    .ToList();

                // Separate out the feedback types below 5%
                var collapsedTypes = feedbackByType
                    .Where(ft => ft.Count < totalFeedback * 0.05) // Below 5%
                    .Select(ft => new FeedbackTypeDetail { OriginalType = ft.Type, Count = ft.Count })
                    .ToList();

                // Filter out the ones that are above 5%
                var feedbackSummary = feedbackByType
                    .Where(ft => ft.Count >= totalFeedback * 0.05)
                    .ToList();

                // Add "Other" category with collapsed types
                if (collapsedTypes.Any())
                {
                    feedbackSummary.Add(new FeedbackTypeSummary
                    {
                        Type = "Other",
                        Count = collapsedTypes.Sum(ct => ct.Count),
                        Details = collapsedTypes
                    });
                }

                return new ServiceHighlight
                {
                    ServiceName = g.Key,
                    TotalFeedback = totalFeedback,
                    DistinctCustomers = g.Select(r => r.CustomerTpid).Distinct().Count(),
                    FeatureRequests = g.Count(r => r.Type == "Feature Request"),
                    Bugs = g.Count(r => r.Type == "Bug"),
                    OverallSentiment = g.Count(r => r.Type == "Feature Request") > g.Count(r => r.Type == "Bug") ? "Positive" : "Neutral",
                    FeedbackTypes = feedbackSummary  // Include the feedback summary with "Other"
                };
            })
            .ToList();

        // Serialize the result to JSON using System.Text.Json
        return JsonSerializer.Serialize(serviceHighlights, new JsonSerializerOptions { WriteIndented = true });
        // Console.WriteLine(json);
    }
}


In [15]:
public List<ServiceHighlight> GenerateSummary(string service)
{
    var filePath = $"{dataRoot}/{service}.csv";
    using (var reader = new StreamReader(filePath))
    using (var csv = new CsvReader(reader, config))
    {
        var records = csv.GetRecords<CSVFeedbackRecord>().ToList();

        // Group by ServiceTree_Name and calculate metrics
        var serviceHighlights = records
            .GroupBy(r => r.ServiceName)
            .Select(g =>
            {
                var totalFeedback = g.Count();
                var feedbackByType = g.GroupBy(r => r.Type)
                                    .Select(t => new FeedbackTypeSummary
                                    {
                                        Type = t.Key,
                                        Count = t.Count()
                                    })
                                    .ToList();

                // Separate out the feedback types below 5%
                var collapsedTypes = feedbackByType
                    .Where(ft => ft.Count < totalFeedback * 0.05) // Below 5%
                    .Select(ft => new FeedbackTypeDetail { OriginalType = ft.Type, Count = ft.Count })
                    .ToList();

                // Filter out the ones that are above 5%
                var feedbackSummary = feedbackByType
                    .Where(ft => ft.Count >= totalFeedback * 0.05)
                    .ToList();

                // Add "Other" category with collapsed types
                if (collapsedTypes.Any())
                {
                    feedbackSummary.Add(new FeedbackTypeSummary
                    {
                        Type = "Other",
                        Count = collapsedTypes.Sum(ct => ct.Count),
                        Details = collapsedTypes
                    });
                }

                return new ServiceHighlight
                {
                    ServiceName = g.Key,
                    TotalFeedback = totalFeedback,
                    DistinctCustomers = g.Select(r => r.CustomerTpid).Distinct().Count(),
                    FeatureRequests = g.Count(r => r.Type == "Feature Request"),
                    Bugs = g.Count(r => r.Type == "Bug"),
                    OverallSentiment = g.Count(r => r.Type == "Feature Request") > g.Count(r => r.Type == "Bug") ? "Positive" : "Neutral",
                    FeedbackTypes = feedbackSummary  // Include the feedback summary with "Other"
                };
            })
            .ToList();

        return serviceHighlights;  // Return the list of ServiceHighlight objects
    }
}

In [16]:
public void GenerateCombinedSummary(string[] services)
{
    var allServiceHighlights = new List<ServiceHighlight>();

    foreach (var service in services)
    {
        // Call GenerateSummary for each service and add to the combined list
        var serviceHighlights = GenerateSummary(service);
        allServiceHighlights.AddRange(serviceHighlights);
    }

    // Serialize the combined result to JSON and write to a file
    var json = JsonSerializer.Serialize(allServiceHighlights, new JsonSerializerOptions { WriteIndented = true });
    var outputFilePath = $"{dataRoot}/combined_service_summary.json";
    File.WriteAllText(outputFilePath, json);

    Console.WriteLine($"Summary generated for all services and saved to {outputFilePath}");
}

In [ ]:
var services = new string[] { "adf", "aks", "cosmosdb" };
GenerateCombinedSummary(services);